# **Study Project:** *Transformer model for prediction of grasping movements*

## Import packages

In [1]:
import os
import cv2
import numpy as np
import IPython.display as ipd
#!pip install ultralytics
#ipd.clear_output()
import ultralytics

## Helper functions

In [2]:
def GetBoundingBoxes(videopath):
    """
    Extract bounding boxes from video.

    Args:
    videopath -- path to video to extract bounding boxes from.
    """
    # Use object tracker to get the bounding boxes and classIDs in a 'results' object
    bbs = np.zeros((1, 7))
    # It is also possible to pass the whole folder as path,
    # but we still want the flexibility to access single videos
    results = model.track(source=videopath, tracker="bytetrack.yaml")
    
    # Get class names
    classes = results[0].names
    
    # Iterate through each frame of a video to get all bounding boxes for a frame
    for frame in range(len(results)):
    
        # x_center, y_center, bbwidth, bbheight of bbs of this frame
        xywh = results[frame].boxes.xywh.detach().cpu().numpy()
        n = len(xywh) # number of bounding boxes
        cls = results[frame].boxes.cls.detach().cpu().numpy().reshape((n,1))
        # if the object tracker is currently tracking at least one object, save the trackingID for that object, else fill with -1 placeholder
        trackingID = results[frame].boxes.id.detach().cpu().numpy().reshape((n,1)) if results[frame].boxes.is_track else np.repeat(-1, n).reshape((n,1))
        frame_count = np.repeat(frame, n).reshape((n,1))
        
        # bind the data together for one frame
        data = np.concatenate((frame_count, cls, xywh, trackingID), axis=1)
        # add all data of this frame to all data of this video
        bbs = np.concatenate((bbs, data), axis=0)
    
    return bbs, classes

In [3]:
def Center(videopath, to_path, bounding_boxes, width=1920, height=1120):

    """
    Create a video where a target object is always centered.

    Args:
    videopath -- path to video to center
    to_path -- location to export centered video to
    bounding_boxes -- bbs of object to center on
    width -- output video dim x, default=1920
    height -- output video dim y, default=1120
    """

    # 1. Read video 
    cap = cv2.VideoCapture(videopath)
    frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    FPS = int(cap.get(cv2.CAP_PROP_FPS))
    
    video = np.zeros((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))
    fc = 0
    ret = True
    
    while (fc < frameCount and ret):
        ret, video[fc] = cap.read()
        fc += 1
    
    cap.release()
    
    
    # 2. Correct video
    bbs = bounding_boxes
    width = width
    height = height
    center_x = (width/2)
    center_y = (height/2)
    x_dists = []
    y_dists = []
    
    for box_pos in bbs:
        if np.isnan(box_pos).any():
            # if all values were nan, then 0 would be the max, so the corrected video would be the original video
            x_dists.append(0)
            y_dists.append(0)
        else:
            # get center of bb
            #center_x_bb = (box_pos[0] + box_pos[2]) / 2
            #center_y_bb = (box_pos[1] + box_pos[3]) / 2
            center_x_bb = box_pos[0]
            center_y_bb = box_pos[1]
            # calculate distances
            x_dists.append(abs(center_x-center_x_bb))
            y_dists.append(abs(center_y-center_y_bb))
    
    
    video_corrected = np.zeros((len(video),
                              int(height + max(y_dists)*2),
                              int(width + max(x_dists)*2),
                              3), dtype=int)
    
    
    # 3. Center video
    # get coords for placement height and width. may be switched
    start_row = (video_corrected.shape[1] - height) // 2
    start_col = (video_corrected.shape[2] - width) // 2
    
    for idx, frame in enumerate(video):
    
        # get matching bb
        box_curr = bbs[idx]
        
        # If there is no information on fruit, color the frame black (skip iter)
        if np.isnan(box_curr).any():
            continue
            # frame[:] = 0
            # frame = np.zeros(frame.shape)
        else:
            # get center of bb
            #center_x_bb = (box_curr[0] + box_curr[2]) / 2
            #center_y_bb = (box_curr[1] + box_curr[3]) / 2
            center_x_bb = box_curr[0]
            center_y_bb = box_curr[1]
            
            # get offset of center
            # pos if bounding box is to right of center, else negative
            x_offset = int(center_x_bb - center_x)
            # pos if bounding box is below center, else negative
            y_offset = int(center_y_bb - center_y)
            
            #!coordinates until here are for old video
            
            #get fitting indices for new video
            fixed_start_row = start_row - y_offset
            fixed_start_col = start_col - x_offset
            
            fixed_end_row = fixed_start_row + height
            fixed_end_col = fixed_start_col + width
        
            # Checkup
            if((fixed_start_row or
                fixed_start_col or
                fixed_end_row or
                fixed_end_col) < 0):
                print("Negative Index!")
            
            # Save centered + corrected in new video
            video_corrected[idx][fixed_start_row:fixed_end_row,
                              fixed_start_col:fixed_end_col] = frame
    
    
    # 4. Save video
    video_corrected = np.uint8(video_corrected)
    
    height_new = int(video_corrected.shape[1])
    width_new = int(video_corrected.shape[2])
    
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(to_path[:-4]+'_centered.mp4', fourcc, FPS, (width_new, height_new), True)
    for idx in range(len(video)):
        out.write(video_corrected[idx])
    out.release()

## Video centering

In [4]:
# Load model
model = ultralytics.YOLO('yolov8n.pt')

# List videos
folderpath = './Trials/'
labelpath = './Labels/'
folder = [f for f in os.listdir(folderpath) if os.path.isfile(os.path.join(folderpath, f))]

# Example video
#ipd.Video(folderpath+'banana02.mp4', width=1920/1.7, height=1120/1.7)

**Procedure:**
1. Get the bounding boxes of all objects in each raw video.
2. Filter for hand and target object bounding boxes.
3. Target centering: fixate the target object (TARGET.mp4) using interpolation.

In [5]:
# Iterate through every video file in the folder
for i, video in enumerate(folder):

    print(f"Video {i+1}/{len(folder)}: {video}\n")
    
    # Get length of input array (number of frames)
    cap = cv2.VideoCapture(folderpath+video)
    frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    
    
    # 1. Get all bbs of this video
    # (x_center, y_center, bbwidth, bbheight, class, trackerID, frame, filename)
    bbs, classes = GetBoundingBoxes(folderpath+video)
    video_count = np.repeat(i, len(bbs)).reshape((len(bbs),1))
    bbs = np.concatenate((video_count, bbs), axis=1)
    
    
    # 2. Filter for hand and target fruit by class
    # Get the target class to filter for as number
    if '.mp4' in video:
        # returns the string name without '00.mp4' ending (could add try-catch)
        target_class = video[:-6]
    
    # Get class number from lookup table
    target_class = list(classes.keys())[list(classes.values()).index(target_class)]
    hand_class = list(classes.keys())[list(classes.values()).index('person')]
    
    # Filter data
    target_bbs = bbs[(bbs[:, 2] == target_class)]
    hand_bbs = bbs[(bbs[:, 2] == hand_class)]
    
    # To Do: Use trackerID to track the same object
    # -> could be added later on, as we only use one object per trial?
    # -> but if we have detection of more than one target fruit, it breaks something
    # because it will try to calculate positions from more than one bb per frame.
    # problem so far: if detection ends for one frame, a new trackerID is assigned
    # -> for this approach to work we would have to compare the position of the
    # last tracked trackerID to each object with newly assigned trackerID and
    # take the closest one, and interpolate position if there are no new trackerIDs detected.
    # for now, we leave it out: remove class and trackingID
    target_bbs = target_bbs[:, [0,1,3,4,5,6]]
    hand_bbs = hand_bbs[:, [0,1,3,4,5,6]]
    
    
    # 3. Interpolation: take last known bounding box positions
    # (x_center_fruit, y_center_fruit, bbwidth_fruit, bbheight_fruit, class, trackerID, frame, filename, x_center_hand, y_center_hand, bbwidth_hand, bbheight_hand)
    vid_input = np.zeros((frameCount, 10))
    vid_input[:] = np.nan # use as 'no information' instead of nan, because nan is a string
    
    for frame in range(len(vid_input)):
        # get information on objects
        target_information = target_bbs[(target_bbs[:, 1] == frame), :]
        hand_information = hand_bbs[(hand_bbs[:, 1] == frame), 2:]
        
        # fill information for fruit
        if (frame in target_bbs[:, 1]) & (target_information.shape[0] == 1): # col 1 is frames
            vid_input[frame, :6] = target_information
        else:
            # if we have a frame without information on the object (object was not detected)
            # we take the information from the last row if it was not the first frame
            if frame == 0:
                vid_input[frame, 1] = frame
                vid_input[frame, 0] = i
                continue
            else:
                # interpolation method: last known information
                # (should be replaced by positional interpolation)
                vid_input[frame, :6] = vid_input[frame-1, :6]
        
        # fill information for hand
        if (frame in hand_bbs[:, 1]) & (hand_information.shape[0] == 1): # col 1 is frames
            """
            Fails in the line below if we have more than one detection for the same object/class per frame.
            current workaround: take last known information (like current interpolation method)
            -> better option: compare to last known information and take closer one
            """
            vid_input[frame, 6:] = hand_information
        else:
            # if we have a frame without information on the object (object was not detected)
            # we take the information from the last row if it was not the first frame
            if frame == 0:
                vid_input[frame, 1] = frame
                vid_input[frame, 0] = i
                continue
            else:
                vid_input[frame, 6:] = vid_input[frame-1, 6:]
            
        vid_input[frame, 1] = frame
        vid_input[frame, 0] = i
    
    # get first frame with information of target (used for input later)
    start_frame = 0
    for i, frame in enumerate(vid_input[:, :6]):
        if np.isnan(frame).any():
            start_frame = i+1
        else:
            break
    
    
    # 4. Center: Center on the correct bounding boxes of the target fruit
    Center(folderpath+video, labelpath+video, vid_input[:, 2:6])

Video 1/6: orange05.mp4





    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/126) /Users/florian/Documents/Studium/Master/Semester 2/Study Project/Local/Trials/orange05.mp4: 384x640 1 orange, 195.4ms
video 1/1 (2/126) /Users/florian/Documents/Studium/Master/Semester 2/Study Project/Local/Trials/orange05.mp4: 384x640 1 orange, 201.8ms
video 1/1 (3/126) /Users/florian/Documents/Studium/Master/Semester 2/Study Project/Local/Trials/orange05.mp4: 384x640 1 orange, 167.9ms
video 1/1 (4/126) /Users/florian/Documents/Studium/

Video 2/6: orange07.mp4




    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/126) /Users/florian/Documents/Studium/Master/Semester 2/Study Project/Local/Trials/orange07.mp4: 384x640 1 orange, 259.5ms
video 1/1 (2/126) /Users/florian/Documents/Studium/Master/Semester 2/Study Project/Local/Trials/orange07.mp4: 384x640 1 orange, 160.7ms
video 1/1 (3/126) /Users/florian/Documents/Studium/Master/Semester 2/Study Project/Local/Trials/orange07.mp4: 384x640 1 orange, 154.6ms
video 1/1 (4/126) /Users/florian/Documents/Studium/M

Video 3/6: orange06.mp4



...


    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/116) /Users/florian/Documents/Studium/Master/Semester 2/Study Project/Local/Trials/banana02.mp4: 384x640 1 orange, 1 bed, 1 tv, 1 laptop, 1 keyboard, 1 cell phone, 168.4ms
video 1/1 (2/116) /Users/florian/Documents/Studium/Master/Semester 2/Study Project/Local/Trials/banana02.mp4: 384x640 1 person, 1 orange, 1 bed, 1 tv, 1 laptop, 1 keyboard, 149.0ms
video 1/1 (3/116) /Users/florian/Documents/Studium/Master/Semester 2/Study Project/Local/Trial

Video 5/6: banana03.mp4




    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/116) /Users/florian/Documents/Studium/Master/Semester 2/Study Project/Local/Trials/banana03.mp4: 384x640 1 orange, 1 bed, 1 tv, 1 laptop, 1 keyboard, 1 cell phone, 200.8ms
video 1/1 (2/116) /Users/florian/Documents/Studium/Master/Semester 2/Study Project/Local/Trials/banana03.mp4: 384x640 1 person, 1 orange, 1 bed, 1 tv, 1 laptop, 1 keyboard, 152.0ms
video 1/1 (3/116) /Users/florian/Documents/Studium/Master/Semester 2/Study Project/Local/Trial

Video 6/6: banana04.mp4




    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/116) /Users/florian/Documents/Studium/Master/Semester 2/Study Project/Local/Trials/banana04.mp4: 384x640 1 orange, 1 bed, 1 tv, 1 laptop, 1 keyboard, 1 cell phone, 169.2ms
video 1/1 (2/116) /Users/florian/Documents/Studium/Master/Semester 2/Study Project/Local/Trials/banana04.mp4: 384x640 1 person, 1 orange, 1 bed, 1 tv, 1 laptop, 1 keyboard, 152.3ms
video 1/1 (3/116) /Users/florian/Documents/Studium/Master/Semester 2/Study Project/Local/Trial